## 데이터 로드

In [1]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [2]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['id', 'plylst_title', 'updt_date', 'like_cnt'], axis=1)
train_data.head()

,tags,songs
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [3]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [4]:
song_data = pd.DataFrame(json_data)
song_data = song_data.drop(['album_name', 'song_gn_gnr_basket'], axis=1)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 열 이름 변경

In [5]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,tags,song_id
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954..."
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427..."
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598..."
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033..."
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,..."


In [6]:
song_data.rename(columns={'id':'song_id', 'song_gn_dtl_gnr_basket': 'gnr'}, inplace=True)
song_data = song_data.astype({'issue_date':'int64'})
song_data.head()

,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket,song_id
0,[GN0901],20140512,2255639,[2727],Feelings,[Various Artists],0
1,"[GN1601, GN1606]",20080421,376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[Murray Perahia],1
2,[GN0901],20180518,4698747,[3361],Solsbury Hill (Remastered 2002),[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[Matoma],3
4,"[GN1802, GN1801]",20110824,2008470,[560160],그남자 그여자,[Jude Law],4


## 데이터 추출

- 500개의 플레이리스트 추출

In [41]:
train_data_sample = train_data[:500]

## 태그 병합

- 같은 노래에 부여된 서로 다른 태그들을 합친다
- 그 결과 동일한 태그 리스트가 거의 모든 노래에 부여되었다

In [42]:
train_data_sample = train_data_sample.explode('song_id', ignore_index=True)
train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [43]:
train_dict = dict()

for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'][i]
    tag = train_data_sample['tags'][i]
    
    if song in train_dict:
        for j in tag:
            train_dict[song].add(j)
    
    else:
        train_dict[song] = set(tag)
        
print(train_dict[157435])

{'걸그룹댄스', 'kpop', '여자아이돌', '댄스', '스트레스해소'}


In [44]:
# Inplace 가 False 로 되어 있어, 중복되는 곡들이 존재했었음. True 로 바꾸어 해결
train_data_sample.drop_duplicates(subset='song_id', keep='first',inplace=True)
train_data_sample.shape

(16674, 2)

In [51]:
for i in range(len(train_data_sample)):
    song = train_data_sample['song_id'].iloc[i]
    train_data_sample['tags'].iloc[i] = list(train_dict[song])

train_data_sample.head()

,tags,song_id
0,[락],525514
1,[락],129701
2,[락],383374
3,[락],562083
4,[락],297861


In [52]:
print(train_data_sample.iloc[500])

tags       [카카오톡, 프로필, 다짐, 소원, 카톡, 기분전환, 여행, 새해, 프로필음악, 소망]
song_id                                              115049
Name: 503, dtype: object


In [81]:
# Song Data 에 Tag 를 추가했다는 뜻을 명확하기 위해 merge -> song_tag_appended 로 수정
# 전체 곡에 대해 고려하기 위해 Outer 속성을 추가 (Tag 가 없는 경우 NaN) -> Memory Error 발생으로 NaN 은 잠시 제외
#song_tag_appended = pd.merge(train_data_sample, song_data, how='outer',on='song_id')
song_tag_appended = pd.merge(train_data_sample, song_data)
song_tag_appended = song_tag_appended.astype({'song_id':'int64'})
song_tag_appended.head()

,tags,song_id,gnr,issue_date,album_id,artist_id_basket,song_name,artist_name_basket
0,[락],525514,"[GN1402, GN1401]",20130506,2200223,[734201],Hey Little Girl,[The Sol]
1,[락],129701,"[GN0901, GN0902, GN1001]",20130917,2201802,[536907],Octagon,[Royal Bangs]
2,[락],383374,"[GN1012, GN1005, GN1001]",19911021,2216938,[166978],The Road,[Honeymoon Suite]
3,[락],562083,"[GN1013, GN0901, GN0902, GN1001]",20000919,43227,[19035],Honeymoon,[Phoenix]
4,[락],297861,"[GN1013, GN0901, GN0902, GN1001]",20050306,303657,[170117],High,[James Blunt]


In [82]:
song_tag_appended.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16674 entries, 0 to 16673
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tags                16674 non-null  object
 1   song_id             16674 non-null  int64 
 2   gnr                 16674 non-null  object
 3   issue_date          16674 non-null  int64 
 4   album_id            16674 non-null  int64 
 5   artist_id_basket    16674 non-null  object
 6   song_name           16674 non-null  object
 7   artist_name_basket  16674 non-null  object
dtypes: int64(3), object(5)
memory usage: 1.1+ MB


## Word2Vec 사용

- 태그 리스트들을 word2vec로 학습시켜 태그 하나와 연관된 다른 태그들을 유추

In [64]:
train_data_sample2 = train_data[:500]

In [65]:
from gensim.models.word2vec import Word2Vec

w2v = Word2Vec(sentences = train_data_sample2['tags'], vector_size = 100, 
               window = 5, min_count = 5, workers = 4, sg = 0)

w2v.wv.vectors.shape

(66, 100)

In [66]:
print(w2v.wv.most_similar('스트레스'))

[('분위기', 0.22454456984996796), ('힙합', 0.17852607369422913), ('국힙', 0.16757138073444366), ('이별', 0.1608145833015442), ('팝', 0.15153270959854126), ('휴식', 0.13615800440311432), ('여름', 0.13451433181762695), ('설렘', 0.11924614757299423), ('추억', 0.11664089560508728), ('회상', 0.11426529288291931)]


## 코사인 유사도 사용

- 세부 장르를 사용해 코사인 유사도 측정한다
- 그후 유사도를 행렬로 저장한다

In [83]:
from sklearn.feature_extraction.text import CountVectorizer

song_tag_appended['gnr_literal'] = song_tag_appended['gnr'].apply(lambda x: (' ').join(x))
count_vect = CountVectorizer()
gnr_mat = count_vect.fit_transform(song_tag_appended['gnr_literal'])

gnr_mat.shape

(16674, 194)

In [84]:
print(gnr_mat.toarray()[1])
print(gnr_mat.toarray()[2])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]


In [92]:
# 여기서 Cosine 유사도 행렬 gnr_sim 은 특정 행의 노래 장르가 각 열의 노래 장르와 얼마나 유사한지를 보여주는 지표
from sklearn.metrics.pairwise import cosine_similarity

gnr_sim = cosine_similarity(gnr_mat, gnr_mat)
gnr_sim[3]

array([0.        , 0.8660254 , 0.28867513, ..., 0.        , 0.        ,
       0.        ])

- 노래 id가 주어지면 유사도 순으로 n개의 노래 추출

In [128]:
# minyear 계산 시 min(minyear, title..) 이 아닌 min(maxyear, title..) 로 되어있던 것 수정
# 최신 노래를 추천하는 것이 맞다고 보는데, 현재 코드는 플레이리스트 내 가장 최신 노래를 기준으로 과거 노래를 탐색하고 있음
import numpy as np

def find_sim_song(df, sim_matrix, songs, top_n=10):
    simi = np.zeros(len(df['song_id']))
    maxyear = 0
    minyear = 3000
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        maxyear = max(maxyear, title_song['issue_date'].values[0]//10000)
        minyear = min(minyear, title_song['issue_date'].values[0]//10000)
    print("maxy: {} / miny: {}".format(maxyear, minyear))
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
        simi = simi + sim_matrix[title_index, :]
    
    simi /= len(songs)
    
    df['similarity'] = simi.reshape(-1, 1)
    temp = df.sort_values(by="similarity", ascending=False)
    print(temp)
    
    for song in songs:
        title_song = df[df['song_id'] == song]
        title_index = title_song.index.values
        
        temp = temp[temp.index.values != title_index]
    
    temp = temp[temp['issue_date'] > minyear*10000]
    temp = temp[temp['issue_date'] < (maxyear+1)*10000]
    
    final_index = temp.index.values[ : top_n]
    
    return df.iloc[final_index]

In [129]:
similar_songs = find_sim_song(song_tag_appended, gnr_sim, [525514, 129701, 229622], 10)
similar_songs[['song_id', 'similarity', 'issue_date']]

maxy: 2013 / miny: 2006
                                             tags  song_id  \
11005                               [휴식, 힐링, 잔잔한]   191430   
15445  [혼자, 감성적인, 추억, 휴가, 잔잔한, 분위기, 주말, 휴식, 책읽을때]   463782   
4912                                 [겨울, 설렘, 사랑]   229622   
9                                             [락]   205238   
10498                                 [힐링, 봄, 카페]   701978   
...                                           ...      ...   
6998                                         [일렉]   484600   
6999                                         [일렉]   443228   
7000             [일렉, 알앤비, 팝송, 느낌있는, 분위기, 비트, 소울]   539755   
7001                                         [일렉]   158580   
16673                                      [뉴에이지]   687152   

                                            gnr  issue_date  album_id  \
11005          [GN1402, GN1401, GN0901, GN0902]    20140407   2249887   
15445          [GN1402, GN1401, GN0901, GN0902]    20170227  10041704   
4912        

,song_id,similarity,issue_date
9,205238,0.606558,20110124
10498,701978,0.575727,20101011
16038,513731,0.569036,20111018
16037,489449,0.569036,20111111
9997,392798,0.569036,20111018
9450,598147,0.569036,20131105
695,310974,0.569036,20061107
16043,138448,0.569036,20130401
9445,233718,0.569036,20130409
8810,117724,0.569036,20110321


## 노래 추천

- w2v로 추출한 태그에 해당하는 플레이리스트
- 세부 장르의 유사도가 높은 노래 리스트
- 히스토리(test 플레이리스트)의 발행 연도와 같은 연도에 발행한 노래

In [23]:
def song_recommand(tags, songs, tag_df, song_df, sim_mat):
    ts = tags
    
    # 태그가 존재할 경우 + 태그의 개수가 3개 미만인경우 w2v로 태그를 5개까지 늘린다
    while len(ts) != 0 and len(ts) < 3:
        for tag in ts:
            for i in range(5):
                stag = w2v.wv.most_similar(tag)[i][0]
                
                if not stag in ts:
                    ts.append(stag)
                
                if len(ts) >= 3:
                    break
            if len(ts) >= 3:
                break
    
    # 해당 태그가 존재하는 플레이리스트의 노래를 추출하고 등장 빈도수로 정렬한다
    tag_songs = dict()
    
    for tag in ts:
        for i in range(len(tag_df['song_id'])):
            if tag in tag_df['tags'][i]:
                
                for ss in tag_df['song_id'][i]:
                    if not ss in songs:
                        
                        if ss in tag_songs:
                            tag_songs[ss] += 1
                            
                        else:
                            tag_songs[ss] = 1
                        
    tag_songs = sorted(tag_songs.items(), key=lambda x: x[1], reverse=True)
    
    # 기존 노래(히스토리)가 있는 경우 장르 유사도를 계산해
    #상위 100개의 노래를 찾아낸다
    if len(songs) > 0:
        simi_songs = find_sim_song(song_df, sim_mat, songs, 100)
    
    # 기존 노래(히스토리)가 없는 경우 최신 노래(2018~2023년도)를 찾아낸다
    else:
        simi_songs = song_df
        for song in songs:
            title_song = song_df[song_df['song_id'] == song]
            title_index = title_song.index.values
        
            simi_songs = simi_songs[simi_songs.index.values != title_index]
    
        simi_songs = simi_songs[simi_songs['issue_date'] > 20180000]
        simi_songs = simi_songs[simi_songs['issue_date'] < 20240000]
    
    # 태그로 만들어낸 플레이리스트와 장르 유사도로 만들어낸 노래 목록
    # 둘 모두에 존재하는 노래 10개 추출한다
    recommanded = []
    index = 0
    
    while len(recommanded) < 10 and index < len(tag_songs):
        tag_song = tag_songs[index][0]
        
        if tag_song in simi_songs:
            recommanded.append(tag_song)
            
        index += 1
        
    # 둘 모두에 존재하는 노래가 10개 미만인 경우
    # 각각에서 우선순위가 높은 노래들을 추출한다   
    if len(recommanded) < 10:
        if len(recommanded) % 2 == 0:
            sc = (10-len(recommanded)) / 2
        else:
            sc = (10-len(recommanded)) // 2
        
        index = 0
        while len(tag_songs) != 0 and len(recommanded) < (10 - sc):
            tag_song = tag_songs[index][0]
            
            if not tag_song in recommanded:
                recommanded.append(tag_song)
            
            index += 1
        
        index = 0
        while len(recommanded) < 10:
            simi_song = simi_songs['song_id'].values[index]
            
            if not simi_song in recommanded:
                recommanded.append(simi_song)
            
            index += 1
            
    # 추출된 노래 id를 가지고 데이터프레임을 추출한다
    rec_index = []
    
    for rec in recommanded:
        title_song = song_df[song_df['song_id'] == rec]
        title_index = title_song.index
        rec_index.append(title_index[0]) # 중복제거를 했음에도 중복이 제거 안됨
    
    return song_df.loc[rec_index]

### 태그와 노래 목록 존재

In [24]:
my_tags1 = ['락']
my_songs1 = [525514, 129701, 229622]
rec1 = song_recommand(my_tags1, my_songs1, train_data_sample2, merge2, gnr_sim)
rec1[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

,song_id,song_name,artist_name_basket,issue_date
1295,146989,YOUTH,[Troye Sivan],20160123
1303,430106,Everglow,[Coldplay],20151204
1342,15124,Something Just Like This,"[The Chainsmokers, Coldplay]",20170407
3514,258814,Surrender (Feat. 린),[챈슬러 (Chancellor)],20161130
3520,86380,오늘도 그대만 (Feat. 정동원),[T.P RETRO (타디스 프로젝트舊)],20170228
13358,598147,Take Your Time,[Louis Yoelin],20131105
13353,233718,Another Day Alone,[Louis Yoelin],20130409
7787,9336,Lego House,[Ed Sheeran],20110909
7781,693335,Broken,[Jake Bugg],20121015
7780,237693,Taste It,[Jake Bugg],20121015


### 노래 목록만 존재

In [25]:
my_tags2 = []
my_songs2 = [525514, 129701, 229622]
rec2 = song_recommand(my_tags2, my_songs2, train_data_sample2, merge2, gnr_sim)
rec2[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

,song_id,song_name,artist_name_basket,issue_date
13358,598147,Take Your Time,[Louis Yoelin],20131105
13353,233718,Another Day Alone,[Louis Yoelin],20130409
7787,9336,Lego House,[Ed Sheeran],20110909
7781,693335,Broken,[Jake Bugg],20121015
7780,237693,Taste It,[Jake Bugg],20121015
7779,682132,Two Fingers,[Jake Bugg],20121015
7778,33428,Lightning Bolt,[Jake Bugg],20121015
7777,443713,Give Me Love,[Ed Sheeran],20111115
1287,310974,9 Crimes,[Damien Rice],20061107
3421,322130,Mrs. Cold,[Kings Of Convenience],20090924


### 태그만 존재

In [26]:
my_tags3 = ['락']
my_songs3 = []
rec3 = song_recommand(my_tags3, my_songs3, train_data_sample2, merge2, gnr_sim)
rec3[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

,song_id,song_name,artist_name_basket,issue_date
1295,146989,YOUTH,[Troye Sivan],20160123
1303,430106,Everglow,[Coldplay],20151204
1342,15124,Something Just Like This,"[The Chainsmokers, Coldplay]",20170407
3514,258814,Surrender (Feat. 린),[챈슬러 (Chancellor)],20161130
3520,86380,오늘도 그대만 (Feat. 정동원),[T.P RETRO (타디스 프로젝트舊)],20170228
125,394031,Into the Unknown (From &#34;Frozen 2&#34;/Soun...,"[Idina Menzel, Aurora]",20191115
183,457519,꿀차,[우효],20180102
184,453762,너 정말 예쁘다,[최낙타],20180410
185,349398,LOVE YA!,[혁오 (HYUKOH)],20180531
186,631142,편지,[장희원],20180603


### 둘 다 없음
- 태그가 없는 경우에도 w2v을 통해 태그가 생성됨

In [27]:
my_tags4 = []
my_songs4 = []
rec4 = song_recommand(my_tags3, my_songs3, train_data_sample2, merge2, gnr_sim)
rec4[['song_id', 'song_name', 'artist_name_basket', 'issue_date']]

,song_id,song_name,artist_name_basket,issue_date
1295,146989,YOUTH,[Troye Sivan],20160123
1303,430106,Everglow,[Coldplay],20151204
1342,15124,Something Just Like This,"[The Chainsmokers, Coldplay]",20170407
3514,258814,Surrender (Feat. 린),[챈슬러 (Chancellor)],20161130
3520,86380,오늘도 그대만 (Feat. 정동원),[T.P RETRO (타디스 프로젝트舊)],20170228
125,394031,Into the Unknown (From &#34;Frozen 2&#34;/Soun...,"[Idina Menzel, Aurora]",20191115
183,457519,꿀차,[우효],20180102
184,453762,너 정말 예쁘다,[최낙타],20180410
185,349398,LOVE YA!,[혁오 (HYUKOH)],20180531
186,631142,편지,[장희원],20180603
